### 简历代码初步实现

<font color="orange">简历类</font>

In [1]:
class Resume
{
    private string _name;
    private string _sex;
    private string _age;
    private string _timeArea;
    private string _company;

    public Resume(string name)
    {
        _name = name;
    }
    //设置个人信息
    public void SetPersonalInfo(string sex, string age)
    {
        _sex = sex;
        _age = age;
    }

    public void SetWorkExperience(string timeArea, string company)
    {
        _timeArea = timeArea;
        _company = company;
    }

    public void Display()
    {
        Console.WriteLine($"{_name} {_sex} {_age}");
        Console.WriteLine($"工作经历：{_timeArea} {_company}");
    }
}

<font color="orange">客户端调用代码</font>

In [2]:
Resume a = new Resume("大鸟");
a.SetPersonalInfo("男", "29");
a.SetWorkExperience("1998-2000", "XX公司");
Resume b = new Resume("大鸟");
b.SetPersonalInfo("男", "29");
b.SetWorkExperience("1998-2000", "XX公司");
Resume c = new Resume("大鸟");
c.SetPersonalInfo("男", "29");
c.SetWorkExperience("1998-2000", "XX公司");

a.Display();
b.Display();
c.Display();

大鸟 男 29
工作经历：1998-2000 XX公司
大鸟 男 29
工作经历：1998-2000 XX公司
大鸟 男 29
工作经历：1998-2000 XX公司


类似于手写简历，三份简历需要三次实例化，导致客户端代码变得麻烦

### 原型模式

> <font color="yellow">原型模式(Prototype): 用原型实例指定创建对象的种类，并且通过拷贝这些原型创建新的对象</font>

In [6]:
---
title: "原型模式（Prototype）结构图"
---
classDiagram
    class Client{
        -prototype
    }
    class Prototype{
        <<interface>>
        +Clone()
    }
    class ConcretePrototype1{
        +Clone()
    }
    class ConcretePrototype2{
        +Clone()
    }
    note for Client "让一个原型克隆自身从而创建一个新的对象"
    note for Prototype "原型类，声明一个克隆自身的接口"
    note for ConcretePrototype1 "具体原型类，实现一个克隆自身的操作"

    Prototype --> Client
    ConcretePrototype1 --|> Prototype
    ConcretePrototype2 --|> Prototype

<font color="orange">原型模式其实就是从一个对象再创建另外一个可定制的对象，而不需要知道任何创建的细节</font>

原型类

In [8]:
abstract class Prototype
{
    private string _id;
    public Prototype(string id)
    {
        _id = id;
    }

    public string Id
    {
        get {return _id;}
    }

    public abstract Prototype Clone();
}

具体原型类

In [9]:
class ConcretePrototype1:Prototype
{
    public ConcretePrototype1(string id): base(id){}

    public override Prototype Clone()
    {
        //创建当前对象的浅表副本。方法是创建一个新对象，然后将当前对象的非静态字段复制到该新对象。
        //如果字段是值类型的，则对该字段进行逐位复制。如果字段是引用类型，则复制引用但不复制引用的对象；
        //因此，原始对象及其副本引用同一对象
        return (Prototype)this.MemberwiseClone();
    }
}

客户端代码

In [10]:
ConcretePrototype1 p1 = new ConcretePrototype1("I");
//克隆类ConcretePrototype1的对象p1就能得到新的实例c1
ConcretePrototype1 c1 = (ConcretePrototype1)p1.Clone();
Console.WriteLine($"Cloned: {c1.Id}");


Cloned: I


通过克隆类的方式就不用实例化ConcretePrototype1了，直接克隆就行了。对于.NET 而言，原型抽象类Prototype是用不着的，因为克隆实在是太常用了，所以.NET在System命名空间中提供了ICloneable接口，其中就有一个唯一的方法Clone()这样就只需要实现这个接口就可以完成原型模式了。

### 简历的原型实现

In [13]:
classDiagram
    class 简历{
        +Clone() object
        +设置个人信息(in sex string, in age string)
        +设置工作经历(in timeArea string, in company string)
        +显示()
    }
    class ICloneable{
        +Clone() object
    }
    ICloneable <|-- 简历

<font color="orange">简历类</font>

In [14]:
class Resume: ICloneable
{
    private string _name;
    private string _sex;
    private string _age;
    private string _timeArea;
    private string _company;

    public Resume(string name)
    {
        _name = name;
    }
    //设置个人信息
    public void SetPersonalInfo(string sex, string age)
    {
        _sex = sex;
        _age = age;
    }

    public void SetWorkExperience(string timeArea, string company)
    {
        _timeArea = timeArea;
        _company = company;
    }

    public void Display()
    {
        Console.WriteLine($"{_name} {_sex} {_age}");
        Console.WriteLine($"工作经历：{_timeArea} {_company}");
    }

    public object Clone()
    {
        return (object)this.MemberwiseClone();
    }
}

<font color="orange">客户端代码</font>

In [15]:
Resume a = new Resume("大鸟");
a.SetPersonalInfo("男", "29");
a.SetWorkExperience("1998-2000", "XX公司");
Resume b = (Resume)a.Clone();
b.SetWorkExperience("1998-2006","YY企业");
Resume c = (Resume)a.Clone();
c.SetPersonalInfo("男","24");

a.Display();
b.Display();
c.Display();

大鸟 男 29
工作经历：1998-2000 XX公司
大鸟 男 29
工作经历：1998-2006 YY企业
大鸟 男 24
工作经历：1998-2000 XX公司


使用原型模式可以让客户端代码清爽很多，如果想改某份简历，只需要对这份简历做一定的修改就可以了，不会影响到其它简历，相同的部分就不用再重复了。并且对性能也会有很大提高，每NEW一次，都需要执行一次构造函数，如果构造函数的执行时间很长，那么多次执行这个初始化操作就实在太低效了。一般在初始化信息不发生变化的额情况下，克隆是最好的办法。这即隐藏了对象创建的细节，又对性能是大大的提高。

### 浅复制与深复制

MemberwiseClone()方法是这样的，创建当前对象的浅表副本。方法是创建一个新对象，然后将当前对象的非静态字段复制到该新对象。如果字段是值类型的，则对该字段进行逐位复制。如果字段是引用类型，则复制引用但不复制引用的对象；因此，原始对象及其副本引用同一对象

In [17]:
classDiagram
    class 工作经历{
        +时间区间 string
        +公司 string
    }
    class 简历{
        +Clone() object
        +设置个人信息(in sex string, in age string)
        +设置工作经历(in timeArea string, in company string)
        +显示()
    }
    class ICloneable{
        +Clone() object
    }
    ICloneable <|-- 简历
    简历 "1" *--> "*" 工作经历

<font color="orange">工作经历类</font>

In [19]:
class WorkExperience
{
    private string workDate;
    public string WorkDate
    {
        get{return workDate;}
        set{workDate = value;}
    }
    private string company;
    public string Company
    {
        get{return company;}
        set{company = value;}
    }
}

<font color="orange">简历类</font>

In [20]:
class Resume: ICloneable
{
    private string _name;
    private string _sex;
    private string _age;
    
    private WorkExperience _work;

    public Resume(string name)
    {
        _name = name;
        _work = new WorkExperience();
    }
    //设置个人信息
    public void SetPersonalInfo(string sex, string age)
    {
        _sex = sex;
        _age = age;
    }

    public void SetWorkExperience(string timeArea, string company)
    {
        _work.WorkDate = timeArea;
        _work.Company = company;
    }

    public void Display()
    {
        Console.WriteLine($"{_name} {_sex} {_age}");
        Console.WriteLine($"工作经历：{_work.WorkDate} {_work.Company}");
    }

    public object Clone()
    {
        return (object)this.MemberwiseClone();
    }
}

<font color="orange">客户端代码</font>

In [21]:
Resume a = new Resume("大鸟");
a.SetPersonalInfo("男", "29");
a.SetWorkExperience("1998-2000", "XX公司");
Resume b = (Resume)a.Clone();
b.SetWorkExperience("1998-2006","YY企业");
Resume c = (Resume)a.Clone();
c.SetPersonalInfo("男","24");

a.Display();
b.Display();
c.Display();

大鸟 男 29
工作经历：1998-2006 YY企业
大鸟 男 29
工作经历：1998-2006 YY企业
大鸟 男 24
工作经历：1998-2006 YY企业


浅复制，被复制对象的所有变量都含有与原来的对象相同的值，而所有的对其他对象的引用都仍指向原来的对象。深复制把引用对象的变量指向复制过得新对象，而不是原有的被引用的对象

### 简历的深复制实现

In [25]:
classDiagram
    class 工作经历{
        +时间区间 string
        +公司 string
        +Clone() object
    }
    class 简历{
        +Clone() object
        +设置个人信息(in sex string, in age string)
        +设置工作经历(in timeArea string, in company string)
        +显示()
    }
    class ICloneable{
        <<interface>>
        +Clone() object
    }
    ICloneable <|-- 简历
    ICloneable <|-- 工作经历
    工作经历 "*" <--* "1" 简历

<font color="orange">工作经历类</font>

In [29]:
class WorkExperience: ICloneable
{
    private string workDate;
    public string WorkDate
    {
        get{return workDate;}
        set{workDate = value;}
    }
    private string company;
    public string Company
    {
        get{return company;}
        set{company = value;}
    }
    //工作经历实现克隆方法
    public object Clone()
    {
        return (object)this.MemberwiseClone();
    }
}

<font color="orange">简历类</font>

In [31]:
class Resume: ICloneable
{
    private string _name;
    private string _sex;
    private string _age;
    
    private WorkExperience _work;

    public Resume(string name)
    {
        _name = name;
        _work = new WorkExperience();
    }
    private Resume(WorkExperience work)
    {
        _work = (WorkExperience)work.Clone();
    }
    //设置个人信息
    public void SetPersonalInfo(string sex, string age)
    {
        _sex = sex;
        _age = age;
    }

    public void SetWorkExperience(string timeArea, string company)
    {
        _work.WorkDate = timeArea;
        _work.Company = company;
    }

    public void Display()
    {
        Console.WriteLine($"{_name} {_sex} {_age}");
        Console.WriteLine($"工作经历：{_work.WorkDate} {_work.Company}");
    }

    public object Clone()
    {
        Resume obj = new Resume(_work);
        obj._name = _name;
        obj._sex = _sex;
        obj._age = _age;
        return obj;
    }
}

<font color="orange">客户端代码</font>

In [32]:
Resume a = new Resume("大鸟");
a.SetPersonalInfo("男", "29");
a.SetWorkExperience("1998-2000", "XX公司");
Resume b = (Resume)a.Clone();
b.SetWorkExperience("1998-2006","YY企业");
Resume c = (Resume)a.Clone();
c.SetPersonalInfo("男","24");

a.Display();
b.Display();
c.Display();

大鸟 男 29
工作经历：1998-2000 XX公司
大鸟 男 29
工作经历：1998-2006 YY企业
大鸟 男 24
工作经历：1998-2000 XX公司
